Write a pyspark dataframe query to find all duplicate emails in a table named Person.

+----+---------+ | Id | Email | +----+---------+ | 1 | a@b.com | | 2 | c@d.com | | 3 | a@b.com | +----+---------+ For example, your query should return the following for the above table:

+---------+ | Email | +---------+ | a@b.com | +---------+


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
spark = SparkSession.builder.appName("Person").getOrCreate()
data = [
    (1, "a@b.com"),
    (2, "c@d.com"),
    (3, "a@b.com")
]
columns = ["Id", "Email"]
df = spark.createDataFrame(data, columns)
df.show()


# Find duplicate emails
df_result = df.groupBy('Email').count().filter('count > 1').select('Email')

# Show the result
df_result.show()

+---+-------+
| Id|  Email|
+---+-------+
|  1|a@b.com|
|  2|c@d.com|
|  3|a@b.com|
+---+-------+

+-------+
|  Email|
+-------+
|a@b.com|
+-------+



In [0]:
##Calculate the total purchase amount for each customer:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum
spark = SparkSession.builder.appName("CustomerPurchaseAnalysis").getOrCreate()
data = [
    (1, 100, "2023-01-15"),
    (2, 150, "2023-02-20"),
    (1, 200, "2023-03-10"),
    (3, 50, "2023-04-05"),
    (2, 120, "2023-05-15"),
    (1, 300, "2023-06-25")
]
columns = ["customer_id", "purchase_amount", "purchase_date"]
df = spark.createDataFrame(data, columns)
df.show()
total_purchase_per_customer = df.groupBy('customer_id').agg(sum('purchase_amount').alias('total_purchase_amount'))
total_purchase_per_customer.show()

+-----------+---------------+-------------+
|customer_id|purchase_amount|purchase_date|
+-----------+---------------+-------------+
|          1|            100|   2023-01-15|
|          2|            150|   2023-02-20|
|          1|            200|   2023-03-10|
|          3|             50|   2023-04-05|
|          2|            120|   2023-05-15|
|          1|            300|   2023-06-25|
+-----------+---------------+-------------+

+-----------+---------------------+
|customer_id|total_purchase_amount|
+-----------+---------------------+
|          1|                  600|
|          2|                  270|
|          3|                   50|
+-----------+---------------------+



In [0]:
#Find the customer with the highest total purchase amount

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, desc
spark = SparkSession.builder.appName("CustomerPurchaseAnalysis").getOrCreate()
data = [
    (1, 100, "2023-01-15"),
    (2, 150, "2023-02-20"),
    (1, 200, "2023-03-10"),
    (3, 50, "2023-04-05"),
    (2, 120, "2023-05-15"),
    (1, 300, "2023-06-25")
]
columns = ["customer_id", "purchase_amount", "purchase_date"]
df = spark.createDataFrame(data, columns)
df.show()
total_purchase_per_customer = df.groupBy('customer_id').agg(sum('purchase_amount').alias('total_purchase_amount'))
customer_with_highest_purchase = total_purchase_per_customer.orderBy(desc('total_purchase_amount')).first()
#the above produces row not dataset so cant use show
customer_with_highest_purchase['customer_id']

+-----------+---------------+-------------+
|customer_id|purchase_amount|purchase_date|
+-----------+---------------+-------------+
|          1|            100|   2023-01-15|
|          2|            150|   2023-02-20|
|          1|            200|   2023-03-10|
|          3|             50|   2023-04-05|
|          2|            120|   2023-05-15|
|          1|            300|   2023-06-25|
+-----------+---------------+-------------+

Out[4]: 1

In [0]:
# Q1.Calculate the total revenue generated from all sales.     
          
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, avg
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("Sales").getOrCreate()
data = [
    (1, "Product A", "Electronics",500,100),
    (2, "Product B", "Clothing",50,200),
    (3, "Product C", "Electronics",800,50),
    (4, "Product D", "Beauty",30,300),
    (5, "Product E", "Clothing",75,150)
]
columns = ["product_id", "product_name", "category","price","quantity_sold"]
df = spark.createDataFrame(data, columns)
df.show()
df.withColumn('total_revenue',col('price')*col('quantity_sold')).agg(sum('total_revenue').alias('total_rev_all_sales')).show()


+----------+------------+-----------+-----+-------------+
|product_id|product_name|   category|price|quantity_sold|
+----------+------------+-----------+-----+-------------+
|         1|   Product A|Electronics|  500|          100|
|         2|   Product B|   Clothing|   50|          200|
|         3|   Product C|Electronics|  800|           50|
|         4|   Product D|     Beauty|   30|          300|
|         5|   Product E|   Clothing|   75|          150|
+----------+------------+-----------+-----+-------------+

+-------------------+
|total_rev_all_sales|
+-------------------+
|             120250|
+-------------------+



In [0]:
## Q2.Find the top 5 best-selling products based on the quantity sold.

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, avg
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("Sales").getOrCreate()
data = [
    (1, "Product A", "Electronics",500,100),
    (2, "Product B", "Clothing",50,200),
    (3, "Product C", "Electronics",800,50),
    (4, "Product D", "Beauty",30,300),
    (5, "Product E", "Clothing",75,150)
]
columns = ["product_id", "product_name", "category","price","quantity_sold"]
df = spark.createDataFrame(data, columns)
# df.show()
df.orderBy(col('quantity_sold').desc()).limit(5).show()

+----------+------------+-----------+-----+-------------+
|product_id|product_name|   category|price|quantity_sold|
+----------+------------+-----------+-----+-------------+
|         4|   Product D|     Beauty|   30|          300|
|         2|   Product B|   Clothing|   50|          200|
|         5|   Product E|   Clothing|   75|          150|
|         1|   Product A|Electronics|  500|          100|
|         3|   Product C|Electronics|  800|           50|
+----------+------------+-----------+-----+-------------+



In [0]:
#Q3.Calculate the average price of products in each category.

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, avg
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("Sales").getOrCreate()
data = [
    (1, "Product A", "Electronics",500,100),
    (2, "Product B", "Clothing",50,200),
    (3, "Product C", "Electronics",800,50),
    (4, "Product D", "Beauty",30,300),
    (5, "Product E", "Clothing",75,150)
]
columns = ["product_id", "product_name", "category","price","quantity_sold"]
df = spark.createDataFrame(data, columns)
df.show()
df.groupBy('category').agg(avg('price').alias('Avg_per_category')).show()

+----------+------------+-----------+-----+-------------+
|product_id|product_name|   category|price|quantity_sold|
+----------+------------+-----------+-----+-------------+
|         1|   Product A|Electronics|  500|          100|
|         2|   Product B|   Clothing|   50|          200|
|         3|   Product C|Electronics|  800|           50|
|         4|   Product D|     Beauty|   30|          300|
|         5|   Product E|   Clothing|   75|          150|
+----------+------------+-----------+-----+-------------+

+-----------+----------------+
|   category|Avg_per_category|
+-----------+----------------+
|Electronics|           650.0|
|   Clothing|            62.5|
|     Beauty|            30.0|
+-----------+----------------+



In [0]:
#Q4.Identify the category with the highest total revenue.

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, avg
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("Sales").getOrCreate()
data = [
    (1, "Product A", "Electronics",500,100),
    (2, "Product B", "Clothing",50,200),
    (3, "Product C", "Electronics",800,50),
    (4, "Product D", "Beauty",30,300),
    (5, "Product E", "Clothing",75,150)
]
columns = ["product_id", "product_name", "category","price","quantity_sold"]
df = spark.createDataFrame(data, columns)
df.show()
category_revenue = df.withColumn('revenue',col('price')*col('quantity_sold')).groupBy('category').agg(sum('revenue').alias('total_revenue'))
max_category_revenue = category_revenue.orderBy(desc('total_revenue')).first()
max_category_revenue['category']

+----------+------------+-----------+-----+-------------+
|product_id|product_name|   category|price|quantity_sold|
+----------+------------+-----------+-----+-------------+
|         1|   Product A|Electronics|  500|          100|
|         2|   Product B|   Clothing|   50|          200|
|         3|   Product C|Electronics|  800|           50|
|         4|   Product D|     Beauty|   30|          300|
|         5|   Product E|   Clothing|   75|          150|
+----------+------------+-----------+-----+-------------+

Out[8]: 'Electronics'

In [0]:
# Calculate the total payroll cost for the company. Calculate the total number of employees in each department. Sample Dataset:
# employee_id,employee_name,department,salary 1,John Doe,Engineering,90000 2,Jane Smith,Marketing,75000 3,Michael Johnson,Engineering, # 105000 4,Emily Davis,Marketing,80000 5,Robert Brown,Engineering,95000 6,Linda Wilson,HR,60000

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, count

# Create a Spark session
spark = SparkSession.builder.appName("EmployeeAnalysis").getOrCreate()

# Load the dataset
data = [
    (1, "John Doe", "Engineering", 90000),
    (2, "Jane Smith", "Marketing", 75000),
    (3, "Michael Johnson", "Engineering", 105000),
    (4, "Emily Davis", "Marketing", 80000),
    (5, "Robert Brown", "Engineering", 95000),
    (6, "Linda Wilson", "HR", 60000)
]
columns = ["employee_id", "employee_name", "department", "salary"]
df = spark.createDataFrame(data, columns)
df.show()
df.select(sum(df.salary).alias('total_payroll')).show()

+-----------+---------------+-----------+------+
|employee_id|  employee_name| department|salary|
+-----------+---------------+-----------+------+
|          1|       John Doe|Engineering| 90000|
|          2|     Jane Smith|  Marketing| 75000|
|          3|Michael Johnson|Engineering|105000|
|          4|    Emily Davis|  Marketing| 80000|
|          5|   Robert Brown|Engineering| 95000|
|          6|   Linda Wilson|         HR| 60000|
+-----------+---------------+-----------+------+

+-------------+
|total_payroll|
+-------------+
|       505000|
+-------------+



In [0]:
# Find the average salary for each department.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, count

# Create a Spark session
spark = SparkSession.builder.appName("EmployeeAnalysis").getOrCreate()

# Load the dataset
data = [
    (1, "John Doe", "Engineering", 90000),
    (2, "Jane Smith", "Marketing", 75000),
    (3, "Michael Johnson", "Engineering", 105000),
    (4, "Emily Davis", "Marketing", 80000),
    (5, "Robert Brown", "Engineering", 95000),
    (6, "Linda Wilson", "HR", 60000)
]
columns = ["employee_id", "employee_name", "department", "salary"]
df = spark.createDataFrame(data, columns)
df.show()
df.groupBy('department').agg(avg('salary').alias('avg_salary_per_dep')).show()

+-----------+---------------+-----------+------+
|employee_id|  employee_name| department|salary|
+-----------+---------------+-----------+------+
|          1|       John Doe|Engineering| 90000|
|          2|     Jane Smith|  Marketing| 75000|
|          3|Michael Johnson|Engineering|105000|
|          4|    Emily Davis|  Marketing| 80000|
|          5|   Robert Brown|Engineering| 95000|
|          6|   Linda Wilson|         HR| 60000|
+-----------+---------------+-----------+------+

+-----------+------------------+
| department|avg_salary_per_dep|
+-----------+------------------+
|Engineering| 96666.66666666667|
|  Marketing|           77500.0|
|         HR|           60000.0|
+-----------+------------------+



In [0]:
#Identify the highest-paid employee and their department.

from pyspark.sql import SparkSession
from pyspark.sql.functions import max, sum, avg, row_number
from pyspark.sql.window import Window

# Create a Spark session
spark = SparkSession.builder.appName("EmployeeAnalysis").getOrCreate()

# Load the dataset
data = [
    (1, "John Doe", "Engineering", 90000),
    (2, "Jane Smith", "Marketing", 75000),
    (3, "Michael Johnson", "Engineering", 105000),
    (4, "Emily Davis", "Marketing", 80000),
    (5, "Robert Brown", "Engineering", 95000),
    (6, "Linda Wilson", "HR", 60000)
]
columns = ["employee_id", "employee_name", "department", "salary"]
df = spark.createDataFrame(data, columns)
df.show()
w = Window().orderBy(col('salary').desc())
df = df.withColumn('row_num', row_number().over(w))
# df.show()
df.filter('row_num == 1').show()

+-----------+---------------+-----------+------+
|employee_id|  employee_name| department|salary|
+-----------+---------------+-----------+------+
|          1|       John Doe|Engineering| 90000|
|          2|     Jane Smith|  Marketing| 75000|
|          3|Michael Johnson|Engineering|105000|
|          4|    Emily Davis|  Marketing| 80000|
|          5|   Robert Brown|Engineering| 95000|
|          6|   Linda Wilson|         HR| 60000|
+-----------+---------------+-----------+------+

+-----------+---------------+-----------+------+-------+
|employee_id|  employee_name| department|salary|row_num|
+-----------+---------------+-----------+------+-------+
|          3|Michael Johnson|Engineering|105000|      1|
+-----------+---------------+-----------+------+-------+



In [0]:
# PySpark Coding Challenge: Analyzing Online Store Orders

# Task: You have a dataset containing information about orders from an online store. Your task is to use PySpark to analyze the data and answer a few questions using aggregate functions.

# Dataset: The dataset is in CSV format and contains the following columns: order_id, customer_id, order_date, total_amount.

# Questions:

# Calculate the total revenue generated from all orders.


from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, count

# Create a Spark session
spark = SparkSession.builder.appName("OnlineStoreAnalysis").getOrCreate()

# Load the dataset
data = [
    (1, "C101", "2023-07-01", 150),
    (2, "C102", "2023-07-02", 200),
    (3, "C101", "2023-07-02", 100),
    (4, "C103", "2023-07-03", 300),
    (5, "C102", "2023-07-04", 250),
    (6, "C101", "2023-07-05", 120)
]
columns = ["order_id", "customer_id", "order_date", "total_amount"]
df = spark.createDataFrame(data, columns)
df.show()
df.select(sum('total_amount').alias('total_revenue')).show()

+--------+-----------+----------+------------+
|order_id|customer_id|order_date|total_amount|
+--------+-----------+----------+------------+
|       1|       C101|2023-07-01|         150|
|       2|       C102|2023-07-02|         200|
|       3|       C101|2023-07-02|         100|
|       4|       C103|2023-07-03|         300|
|       5|       C102|2023-07-04|         250|
|       6|       C101|2023-07-05|         120|
+--------+-----------+----------+------------+

+-------------+
|total_revenue|
+-------------+
|         1120|
+-------------+



In [0]:
# Find the average order amount.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, count

# Create a Spark session
spark = SparkSession.builder.appName("OnlineStoreAnalysis").getOrCreate()

# Load the dataset
data = [
    (1, "C101", "2023-07-01", 150),
    (2, "C102", "2023-07-02", 200),
    (3, "C101", "2023-07-02", 100),
    (4, "C103", "2023-07-03", 300),
    (5, "C102", "2023-07-04", 250),
    (6, "C101", "2023-07-05", 120)
]
columns = ["order_id", "customer_id", "order_date", "total_amount"]
df = spark.createDataFrame(data, columns)
df.show()
df.select(avg('total_amount').alias('avg_order_amt')).show()

+--------+-----------+----------+------------+
|order_id|customer_id|order_date|total_amount|
+--------+-----------+----------+------------+
|       1|       C101|2023-07-01|         150|
|       2|       C102|2023-07-02|         200|
|       3|       C101|2023-07-02|         100|
|       4|       C103|2023-07-03|         300|
|       5|       C102|2023-07-04|         250|
|       6|       C101|2023-07-05|         120|
+--------+-----------+----------+------------+

+------------------+
|     avg_order_amt|
+------------------+
|186.66666666666666|
+------------------+



In [0]:
# Identify the highest total order amount and its corresponding customer.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, count
from pyspark.sql.window import Window

# Create a Spark session
spark = SparkSession.builder.appName("OnlineStoreAnalysis").getOrCreate()

# Load the dataset
data = [
    (1, "C101", "2023-07-01", 150),
    (2, "C102", "2023-07-02", 200),
    (3, "C101", "2023-07-02", 100),
    (4, "C103", "2023-07-03", 300),
    (5, "C102", "2023-07-04", 250),
    (6, "C101", "2023-07-05", 120)
]
columns = ["order_id", "customer_id", "order_date", "total_amount"]
df = spark.createDataFrame(data, columns)
df.show()
w = Window().orderBy(col('total_amount').desc())
df = df.withColumn('row_num', row_number().over(w))
# df.show()
df.filter('row_num == 1').show()

+--------+-----------+----------+------------+
|order_id|customer_id|order_date|total_amount|
+--------+-----------+----------+------------+
|       1|       C101|2023-07-01|         150|
|       2|       C102|2023-07-02|         200|
|       3|       C101|2023-07-02|         100|
|       4|       C103|2023-07-03|         300|
|       5|       C102|2023-07-04|         250|
|       6|       C101|2023-07-05|         120|
+--------+-----------+----------+------------+

+--------+-----------+----------+------------+-------+
|order_id|customer_id|order_date|total_amount|row_num|
+--------+-----------+----------+------------+-------+
|       4|       C103|2023-07-03|         300|      1|
+--------+-----------+----------+------------+-------+



In [0]:
# Calculate the total number of orders for each customer.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, count
from pyspark.sql.window import Window

# Create a Spark session
spark = SparkSession.builder.appName("OnlineStoreAnalysis").getOrCreate()

# Load the dataset
data = [
    (1, "C101", "2023-07-01", 150),
    (2, "C102", "2023-07-02", 200),
    (3, "C101", "2023-07-02", 100),
    (4, "C103", "2023-07-03", 300),
    (5, "C102", "2023-07-04", 250),
    (6, "C101", "2023-07-05", 120)
]
columns = ["order_id", "customer_id", "order_date", "total_amount"]
df = spark.createDataFrame(data, columns)
df.show()
df.groupBy('customer_id').agg(count('order_id').alias('total_orders_per_cust')).show()

+--------+-----------+----------+------------+
|order_id|customer_id|order_date|total_amount|
+--------+-----------+----------+------------+
|       1|       C101|2023-07-01|         150|
|       2|       C102|2023-07-02|         200|
|       3|       C101|2023-07-02|         100|
|       4|       C103|2023-07-03|         300|
|       5|       C102|2023-07-04|         250|
|       6|       C101|2023-07-05|         120|
+--------+-----------+----------+------------+

+-----------+---------------------+
|customer_id|total_orders_per_cust|
+-----------+---------------------+
|       C101|                    3|
|       C102|                    2|
|       C103|                    1|
+-----------+---------------------+



In [0]:
# DAY 6: 💡Task: You have a dataset containing student exam scores. Your task is to use PySpark to analyze the data and answer a few questions using aggregate functions.

# 💡Dataset: The dataset is in CSV format and contains the following columns: student_id, subject, score.

# Sample Dataset :

# student_id,subject,score 1,Math,85 2,Science,92 3,Math,78 4,English,88 5,Science,95 6,Math,90

# 💡𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐚𝐯𝐞𝐫𝐚𝐠𝐞 𝐬𝐜𝐨𝐫𝐞 𝐟𝐨𝐫 𝐞𝐚𝐜𝐡 𝐬𝐮𝐛𝐣𝐞𝐜𝐭.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, count
from pyspark.sql.window import Window

# Create a Spark session
spark = SparkSession.builder.appName("StudentExamScores").getOrCreate()

# Load the dataset
data = [
    (1, "Maths", 85),
    (2, "Science", 92),
    (3, "Maths", 78),
    (4, "English", 88),
    (5, "Science", 95),
    (6, "Maths", 90)
]
columns = ["student_id", "subject", "score"]
df = spark.createDataFrame(data, columns)
df.show()
df.groupBy('subject').agg(avg('score').alias('avg_per_subject')).show()

+----------+-------+-----+
|student_id|subject|score|
+----------+-------+-----+
|         1|  Maths|   85|
|         2|Science|   92|
|         3|  Maths|   78|
|         4|English|   88|
|         5|Science|   95|
|         6|  Maths|   90|
+----------+-------+-----+

+-------+-----------------+
|subject|  avg_per_subject|
+-------+-----------------+
|  Maths|84.33333333333333|
|Science|             93.5|
|English|             88.0|
+-------+-----------------+



In [0]:
# 𝐈𝐝𝐞𝐧𝐭𝐢𝐟𝐲 𝐭𝐡𝐞 𝐡𝐢𝐠𝐡𝐞𝐬𝐭 𝐬𝐜𝐨𝐫𝐞 𝐚𝐧𝐝 𝐢𝐭𝐬 𝐜𝐨𝐫𝐫𝐞𝐬𝐩𝐨𝐧𝐝𝐢𝐧𝐠 𝐬𝐭𝐮𝐝𝐞𝐧𝐭 𝐟𝐨𝐫 𝐞𝐚𝐜𝐡 𝐬𝐮𝐛𝐣𝐞𝐜𝐭.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# Create a Spark session
spark = SparkSession.builder.appName("StudentExamScores").getOrCreate()

# Load the dataset
data = [
    (1, "Maths", 85),
    (2, "Science", 92),
    (3, "Maths", 78),
    (4, "English", 88),
    (5, "Science", 95),
    (6, "Maths", 90)
]
columns = ["student_id", "subject", "score"]
df = spark.createDataFrame(data, columns)
df.show()
highest_score_per_subject = df.groupBy("subject").agg(max("score").alias("highest_score"))
# highest_score_per_subject.show()
highest_score_students = df.join(highest_score_per_subject, on="subject").filter(col("score") == col("highest_score"))
highest_score_students.show()

+----------+-------+-----+
|student_id|subject|score|
+----------+-------+-----+
|         1|  Maths|   85|
|         2|Science|   92|
|         3|  Maths|   78|
|         4|English|   88|
|         5|Science|   95|
|         6|  Maths|   90|
+----------+-------+-----+

+-------+----------+-----+-------------+
|subject|student_id|score|highest_score|
+-------+----------+-----+-------------+
|English|         4|   88|           88|
|Science|         5|   95|           95|
|  Maths|         6|   90|           90|
+-------+----------+-----+-------------+



In [0]:
# 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐭𝐨𝐭𝐚𝐥 𝐧𝐮𝐦𝐛𝐞𝐫 𝐨𝐟 𝐬𝐭𝐮𝐝𝐞𝐧𝐭𝐬 𝐰𝐡𝐨 𝐭𝐨𝐨𝐤 𝐞𝐚𝐜𝐡 𝐬𝐮𝐛𝐣𝐞𝐜𝐭.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, count
from pyspark.sql.window import Window

# Create a Spark session
spark = SparkSession.builder.appName("StudentExamScores").getOrCreate()

# Load the dataset
data = [
    (1, "Maths", 85),
    (2, "Science", 92),
    (3, "Maths", 78),
    (4, "English", 88),
    (5, "Science", 95),
    (6, "Maths", 90)
]
columns = ["student_id", "subject", "score"]
df = spark.createDataFrame(data, columns)
df.show()
df.groupBy('subject').agg(count('student_id').alias('total_students_per_sub')).show()

+----------+-------+-----+
|student_id|subject|score|
+----------+-------+-----+
|         1|  Maths|   85|
|         2|Science|   92|
|         3|  Maths|   78|
|         4|English|   88|
|         5|Science|   95|
|         6|  Maths|   90|
+----------+-------+-----+

+-------+----------------------+
|subject|total_students_per_sub|
+-------+----------------------+
|  Maths|                     3|
|Science|                     2|
|English|                     1|
+-------+----------------------+



In [0]:
# 💡𝐅𝐢𝐧𝐝 𝐭𝐡𝐞 𝐬𝐮𝐛𝐣𝐞𝐜𝐭(𝐬) 𝐰𝐢𝐭𝐡 𝐭𝐡𝐞 𝐡𝐢𝐠𝐡𝐞𝐬𝐭 𝐚𝐯𝐞𝐫𝐚𝐠𝐞 𝐬𝐜𝐨𝐫𝐞.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, count
from pyspark.sql.window import Window

# Create a Spark session
spark = SparkSession.builder.appName("StudentExamScores").getOrCreate()

# Load the dataset
data = [
    (1, "Maths", 85),
    (2, "Science", 92),
    (3, "Maths", 78),
    (4, "English", 88),
    (5, "Science", 95),
    (6, "Maths", 90)
]
columns = ["student_id", "subject", "score"]
df = spark.createDataFrame(data, columns)
df.show()
df = df.groupBy('subject').agg(avg('score').alias('avg_per_subject'))
max_avg = df.orderBy(col('avg_per_subject').desc()).first()
max_avg['subject']


+----------+-------+-----+
|student_id|subject|score|
+----------+-------+-----+
|         1|  Maths|   85|
|         2|Science|   92|
|         3|  Maths|   78|
|         4|English|   88|
|         5|Science|   95|
|         6|  Maths|   90|
+----------+-------+-----+

Out[19]: 'Science'

In [0]:
# Questions:1 Sample Dataset :

# columns = ["user_id", "timestamp"]

# data= [("user1", "2023-08-21 10:00:00"), ("user2", "2023-08-21 11:30:00"), ("user1", "2023-08-21 12:15:00"), ("user3", "2023-08-21 13:45:00"), ("user2", "2023-08-21 14:30:00"), ("user1", "2023-08-21 15:00:00")]

# 💡 𝐅𝐢𝐧𝐝 𝐭𝐡𝐞 𝐞𝐚𝐫𝐥𝐢𝐞𝐬𝐭 𝐚𝐧𝐝 𝐥𝐚𝐭𝐞𝐬𝐭 𝐭𝐢𝐦𝐞𝐬𝐭𝐚𝐦𝐩𝐬 𝐢𝐧 𝐭𝐡𝐞 𝐝𝐚𝐭𝐚𝐬𝐞𝐭.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, to_timestamp, min, max
from pyspark.sql.window import Window

# Create a Spark session
spark = SparkSession.builder.appName("StudentExamScores").getOrCreate()

# Load the dataset
data= [
    ("user1", "2023-08-21 10:00:00"),
    ("user2", "2023-08-21 11:30:00"),
    ("user1", "2023-08-21 12:15:00"),
    ("user3", "2023-08-21 13:45:00"),
    ("user2", "2023-08-21 14:30:00"),
    ("user1", "2023-08-21 15:00:00")
    ]
columns = ["user_id", "timestamp"]
df = spark.createDataFrame(data, columns)
df = df.withColumn('timestamp', to_timestamp(col('timestamp')))
df.show()
earliest_timestamp = df.select(min('timestamp').alias('earliest_timestamp'))
earliest_timestamp.show()
latest_timestamp = df.select(max('timestamp').alias('latest_timestamp'))
latest_timestamp.show()

+-------+-------------------+
|user_id|          timestamp|
+-------+-------------------+
|  user1|2023-08-21 10:00:00|
|  user2|2023-08-21 11:30:00|
|  user1|2023-08-21 12:15:00|
|  user3|2023-08-21 13:45:00|
|  user2|2023-08-21 14:30:00|
|  user1|2023-08-21 15:00:00|
+-------+-------------------+

+-------------------+
| earliest_timestamp|
+-------------------+
|2023-08-21 10:00:00|
+-------------------+

+-------------------+
|   latest_timestamp|
+-------------------+
|2023-08-21 15:00:00|
+-------------------+



In [0]:
# 💡 𝐂𝐨𝐮𝐧𝐭 𝐭𝐡𝐞 𝐧𝐮𝐦𝐛𝐞𝐫 𝐨𝐟 𝐚𝐜𝐭𝐢𝐯𝐢𝐭𝐢𝐞𝐬 𝐩𝐞𝐫 𝐮𝐬𝐞𝐫.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, count
from pyspark.sql.window import Window

# Create a Spark session
spark = SparkSession.builder.appName("StudentExamScores").getOrCreate()

# Load the dataset
data= [
    ("user1", "2023-08-21 10:00:00"),
    ("user2", "2023-08-21 11:30:00"),
    ("user1", "2023-08-21 12:15:00"),
    ("user3", "2023-08-21 13:45:00"),
    ("user2", "2023-08-21 14:30:00"),
    ("user1", "2023-08-21 15:00:00")
    ]
columns = ["user_id", "timestamp"]
df = spark.createDataFrame(data, columns)
df = df.withColumn('timestamp', to_timestamp(col('timestamp')))
df.show()
df.groupBy('user_id').agg(count('timestamp').alias('no_of_activities')).show()

+-------+-------------------+
|user_id|          timestamp|
+-------+-------------------+
|  user1|2023-08-21 10:00:00|
|  user2|2023-08-21 11:30:00|
|  user1|2023-08-21 12:15:00|
|  user3|2023-08-21 13:45:00|
|  user2|2023-08-21 14:30:00|
|  user1|2023-08-21 15:00:00|
+-------+-------------------+

+-------+----------------+
|user_id|no_of_activities|
+-------+----------------+
|  user1|               3|
|  user2|               2|
|  user3|               1|
+-------+----------------+



In [0]:
# 💡𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐭𝐢𝐦𝐞 𝐝𝐮𝐫𝐚𝐭𝐢𝐨𝐧 𝐛𝐞𝐭𝐰𝐞𝐞𝐧 𝐜𝐨𝐧𝐬𝐞𝐜𝐮𝐭𝐢𝐯𝐞 𝐚𝐜𝐭𝐢𝐯𝐢𝐭𝐢𝐞𝐬 𝐟𝐨𝐫 𝐞𝐚𝐜𝐡 𝐮𝐬𝐞𝐫.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, count, lag, cast
from pyspark.sql.window import Window

# Create a Spark session
spark = SparkSession.builder.appName("StudentExamScores").getOrCreate()

# Load the dataset
data= [
    ("user1", "2023-08-21 10:00:00"),
    ("user2", "2023-08-21 11:30:00"),
    ("user1", "2023-08-21 12:15:00"),
    ("user3", "2023-08-21 13:45:00"),
    ("user2", "2023-08-21 14:30:00"),
    ("user1", "2023-08-21 15:00:00")
    ]
columns = ["user_id", "timestamp"]
df = spark.createDataFrame(data, columns)
df = df.withColumn('timestamp', to_timestamp(col('timestamp')))
df.show()
# print(df.dtypes)
# print(df.schema)
w = Window().partitionBy('user_id').orderBy('timestamp')
df_with_prev_timestamp = df.withColumn('prev_timestamp',lag('timestamp').over(w))
df_with_prev_timestamp.show()
df_with_prev_timestamp.withColumn('time_duration',col('timestamp').cast('long')- col('prev_timestamp').cast('long')).show()

+-------+-------------------+
|user_id|          timestamp|
+-------+-------------------+
|  user1|2023-08-21 10:00:00|
|  user2|2023-08-21 11:30:00|
|  user1|2023-08-21 12:15:00|
|  user3|2023-08-21 13:45:00|
|  user2|2023-08-21 14:30:00|
|  user1|2023-08-21 15:00:00|
+-------+-------------------+

+-------+-------------------+-------------------+
|user_id|          timestamp|     prev_timestamp|
+-------+-------------------+-------------------+
|  user1|2023-08-21 10:00:00|               null|
|  user1|2023-08-21 12:15:00|2023-08-21 10:00:00|
|  user1|2023-08-21 15:00:00|2023-08-21 12:15:00|
|  user2|2023-08-21 11:30:00|               null|
|  user2|2023-08-21 14:30:00|2023-08-21 11:30:00|
|  user3|2023-08-21 13:45:00|               null|
+-------+-------------------+-------------------+

+-------+-------------------+-------------------+-------------+
|user_id|          timestamp|     prev_timestamp|time_duration|
+-------+-------------------+-------------------+-------------+
|  use

In [0]:
# Questions:2

# Count the number of action
# 2: What are the unique actions recorded in the dataset?
# 3: Calculate the time duration between consecutive activities for each user
# data = [ (1, "login", "2023-08-20 10:23:45"), (2, "view", "2023-08-20 11:15:30"), (1, "purchase", "2023-08-20 12:45:18"), (3, "view", "2023-08-20 13:30:22") ] columns = ["user_id", "action", "timestamp"]

#import relevant libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,FloatType
from pyspark.sql.window import Window
from pyspark.sql.functions import rank,col,min,max,row_number,max,asc,desc,when,count,sum
#Let's create Sparksession First


spark=SparkSession.builder.master("local[1]").appName("Actions").getOrCreate()

data = [
    (1, "login", "2023-08-20 10:23:45"),
    (2, "view", "2023-08-20 11:15:30"),
    (1, "purchase", "2023-08-20 12:45:18"),
    (3, "view", "2023-08-20 13:30:22")
]
columns = ["user_id", "action", "timestamp"]
#Schema

columns = StructType([StructField("user_id",StringType()), StructField("action",StringType()),StructField("timestamp", StringType())])
action_df = spark.createDataFrame(data, columns)
action_df = action_df.withColumn('timestamp', to_timestamp(col('timestamp')))
action_df.show(truncate=False)

#1
action_df.select(count('action')).show()
#2
action_df.select(col('action')).distinct().show()
#3
w = Window().partitionBy('user_id').orderBy('timestamp')
df_with_prev_timestamp = action_df.withColumn('prev_timestamp',lag('timestamp').over(w))
df_with_prev_timestamp.show()
df_with_prev_timestamp.withColumn('time_duration',col('timestamp').cast('long')- col('prev_timestamp').cast('long')).show()


+-------+--------+-------------------+
|user_id|action  |timestamp          |
+-------+--------+-------------------+
|1      |login   |2023-08-20 10:23:45|
|2      |view    |2023-08-20 11:15:30|
|1      |purchase|2023-08-20 12:45:18|
|3      |view    |2023-08-20 13:30:22|
+-------+--------+-------------------+

+-------------+
|count(action)|
+-------------+
|            4|
+-------------+

+--------+
|  action|
+--------+
|   login|
|    view|
|purchase|
+--------+

+-------+--------+-------------------+-------------------+
|user_id|  action|          timestamp|     prev_timestamp|
+-------+--------+-------------------+-------------------+
|      1|   login|2023-08-20 10:23:45|               null|
|      1|purchase|2023-08-20 12:45:18|2023-08-20 10:23:45|
|      2|    view|2023-08-20 11:15:30|               null|
|      3|    view|2023-08-20 13:30:22|               null|
+-------+--------+-------------------+-------------------+

+-------+--------+-------------------+-----------------

In [0]:
# Q2 . Find out min, max and cumulative salary in a dataset with emp_name, dept_name and salary as columns

# some interviewer will specifically ask you to Create a sparksession, dataframe , define schema and then write the logic.

# (Asked in Service based companies)

# Best Answer 

#import relevant libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,FloatType
from pyspark.sql.window import Window
from pyspark.sql.functions import rank,col,min,max,row_number,max,asc,desc,when,count,sum, cume_dist
#Let's create Sparksession First


spark=SparkSession.builder.master("local[1]").appName("Prep").getOrCreate()

data= [("James", "Sales", 2000),
("sofy", "Sales", 3000),
("Laren", "Sales", 4000),
("Kiku", "Sales", 5000),
("Sam", "Finance", 6000),
("Samuel", "Finance", 7000),
("Yash", "Finance", 8000),
("Rabin", "Finance", 9000),
("Lukasz", "Marketing", 10000),
("Jolly", "Marketing", 11000),
("Mausam", "Marketing", 12000),
("Lamba", "Marketing", 13000),
("Jogesh", "HR", 14000),
("Mannu", "HR", 15000),
("Sylvia", "HR", 16000),
("Sama", "HR", 17000),
]

#Schema

emp_schema = StructType([StructField("name",StringType()), StructField("dept_name",StringType()),StructField("Salary", StringType())])
employees_Salary_df = spark.createDataFrame(data, emp_schema)
employees_Salary_df.show(truncate=False)

#creating window
w = Window().partitionBy('dept_name').orderBy('Salary')




#getting cumulative salary
employees_Salary_df.withColumn('cumu_dist',cume_dist().over(w)).show()


#getting min and max salary
employees_Salary_df.groupBy('dept_name').agg(min('Salary').alias('min_salary'),max('Salary').alias('max_salary')).show()



+------+---------+------+
|name  |dept_name|Salary|
+------+---------+------+
|James |Sales    |2000  |
|sofy  |Sales    |3000  |
|Laren |Sales    |4000  |
|Kiku  |Sales    |5000  |
|Sam   |Finance  |6000  |
|Samuel|Finance  |7000  |
|Yash  |Finance  |8000  |
|Rabin |Finance  |9000  |
|Lukasz|Marketing|10000 |
|Jolly |Marketing|11000 |
|Mausam|Marketing|12000 |
|Lamba |Marketing|13000 |
|Jogesh|HR       |14000 |
|Mannu |HR       |15000 |
|Sylvia|HR       |16000 |
|Sama  |HR       |17000 |
+------+---------+------+

+------+---------+------+---------+
|  name|dept_name|Salary|cumu_dist|
+------+---------+------+---------+
|   Sam|  Finance|  6000|     0.25|
|Samuel|  Finance|  7000|      0.5|
|  Yash|  Finance|  8000|     0.75|
| Rabin|  Finance|  9000|      1.0|
|Jogesh|       HR| 14000|     0.25|
| Mannu|       HR| 15000|      0.5|
|Sylvia|       HR| 16000|     0.75|
|  Sama|       HR| 17000|      1.0|
|Lukasz|Marketing| 10000|     0.25|
| Jolly|Marketing| 11000|      0.5|
|Mausam|Mar


🌟 Pyspark scenario questions : 🤷‍♂️
==================
👉 Here in this question we used LAG(), datediff() and Aggregate MAX() functions.

✔ LAG() function is a window function that returns the value that is offset rows before the current row, and defaults if there are less than offset rows before the current row. This is equivalent to the LAG function in SQL. The PySpark Window functions operate on a group of rows (like frame, partition) and return a single value for every input row.

Imagine a vast customer interaction dataset, and you want to understand the maximum time gap between consecutive interactions for each customer. This intricate task can be efficiently accomplished using PySpark's LAG function in conjunction with the Aggregate MAX function.

🔅 Challenge:
------------
Calculate the longest time duration between two consecutive customer interactions using the PySpark LAG function. Moreover, you want to find the maximum of these time gaps using the Aggregate MAX function. This intricate analysis can offer valuable insights into customer engagement patterns.

🔑 Solution :
----------
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("ComplexFunctionExample").getOrCreate()

hashtag#sample  customer interaction data
data = [("Cust1", "2023-07-01", "Interaction"),
 ("Cust1", "2023-07-03", "Interaction"),
 ("Cust1", "2023-07-05", "Interaction"),
 ("Cust2", "2023-07-02", "Interaction"),
 ("Cust2", "2023-07-06", "Interaction")]

hashtag#create  Dataframe
columns = ["customer_id","interaction_date","interaction_type"]
df = spark.createDataFrame(data,columns)


output:
=====
+-----------+------------+
|customer_id|max_time_gap|
+-----------+------------+
|   Cust2|      4|
|   Cust1|      2|
+-----------+------------+

📈 Logic :- we used LAG() function to compare the interaction of previous customer at specified window, used datediff() to get the difference between interactions, Aggregation MAX() to pull out the maximum time gap.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,FloatType
from pyspark.sql.window import Window
import pyspark.sql.functions as F

spark = SparkSession.builder.master("local[1]").appName("ComplexFunctionExample").getOrCreate()

#sample  customer interaction data
data = [("Cust1", "2023-07-01", "Interaction"),
 ("Cust1", "2023-07-03", "Interaction"),
 ("Cust1", "2023-07-05", "Interaction"),
 ("Cust2", "2023-07-02", "Interaction"),
 ("Cust2", "2023-07-06", "Interaction")]

#create  Dataframe

schema = StructType([StructField("customer_id",StringType()), StructField("interaction_date",StringType()),StructField("interaction_type", StringType())])
employees_Salary_df = spark.createDataFrame(data, schema)
employees_Salary_df.show(truncate=False)

w = Window().partitionBy('customer_id').orderBy('interaction_date')
df_prev_date = employees_Salary_df.withColumn('prev_date',F.lag('interaction_date').over(w))
df_date_diff = df_prev_date.withColumn('diff_date',F.datediff(col('interaction_date'),col('prev_date')))
df_date_diff.groupBy('customer_id').agg(max('diff_date').alias('maximum time gap')).show()

+-----------+----------------+----------------+
|customer_id|interaction_date|interaction_type|
+-----------+----------------+----------------+
|Cust1      |2023-07-01      |Interaction     |
|Cust1      |2023-07-03      |Interaction     |
|Cust1      |2023-07-05      |Interaction     |
|Cust2      |2023-07-02      |Interaction     |
|Cust2      |2023-07-06      |Interaction     |
+-----------+----------------+----------------+

+-----------+----------------+
|customer_id|maximum time gap|
+-----------+----------------+
|      Cust1|               2|
|      Cust2|               4|
+-----------+----------------+



In [0]:
# Write a Pyspark query to find the overall acceptance rate of requests, which is the number of acceptance divided by the number of requests. Return the answer rounded to 2 decimals places. If there are duplicate requests consider them only.If there are no requests at all, you should return 0.00 as the accept_rate.

# FriendRequest table: +-----------+------------+--------------+ | sender_id | send_to_id | request_date | +-----------+------------+--------------+ | 1 | 2 | 2016/06/01 | | 1 | 3 | 2016/06/01 | | 1 | 3 | 2016/06/01 | +-----------+------------+--------------+

# RequestAccepted table: +--------------+-------------+-------------+ | requester_id | accepter_id | accept_date | +--------------+-------------+-------------+ | 1 | 2 | 2016/06/03 | | 1 | 3 | 2016/06/08 | +--------------+-------------+-------------+

# Result table: +-------------+ | unique_accepted_request | +-------------+ | 2 | +-------------+

# +-------------+ | total_request | +-------------+ | 3 | +-------------+


from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, count

# Create a Spark session
spark = SparkSession.builder.appName("FriendRequest").getOrCreate()
spark1 = SparkSession.builder.appName("RequestAccepted").getOrCreate()

# Load the dataset
data = [
    (1, 2, "2016/06/01"),
    (1, 3, "2016/06/01"),
    (1, 3, "2016/06/01")
]
data1 = [
    (1, 2, "2016/06/03"),
    (1, 3, "2016/06/08")
]
columns = ["sender_id", "send_to_id", "request_date"]
columns1 = ["requester_id", "accepter_id", "accept_date"]
df_FR = spark.createDataFrame(data, columns)
df_RA = spark.createDataFrame(data1, columns1)
# df_FR.show()
# df_RA.show()

# Step 01: Join data frames based on conditions
joined_df = df_FR.join(df_RA, (df_FR.sender_id == df_RA.requester_id) & (df_FR.send_to_id == df_RA.accepter_id),'inner')
joined_df.show()
# Step 02: Total requests and unique accepted requests
total_request = joined_df.count()
print('total_requests :', total_request)
unique_requests = joined_df.select('requester_id','accepter_id').distinct().count()
print('unique_requests :',unique_requests)
# Step 03: Calculate the acceptance rate
accepatance_rate = 0 if total_request == 0 else round(unique_requests/total_request,2)
# Step 04: Show the result
result_df = spark.createDataFrame([(total_request, unique_requests, accepatance_rate)],['total_request', 'unique_requests', 'accepatance_rate'])
result_df.show()

+---------+----------+------------+------------+-----------+-----------+
|sender_id|send_to_id|request_date|requester_id|accepter_id|accept_date|
+---------+----------+------------+------------+-----------+-----------+
|        1|         2|  2016/06/01|           1|          2| 2016/06/03|
|        1|         3|  2016/06/01|           1|          3| 2016/06/08|
|        1|         3|  2016/06/01|           1|          3| 2016/06/08|
+---------+----------+------------+------------+-----------+-----------+

total_requests : 3
unique_requests : 2
+-------------+---------------+----------------+
|total_request|unique_requests|accepatance_rate|
+-------------+---------------+----------------+
|            3|              2|            0.67|
+-------------+---------------+----------------+



Every Advance Function
You have a dataset containing user activity logs in a PySpark DataFrame with the following columns: user_id, timestamp, and action. The action column indicates whether the user started or ended a session. It can have values 'start' or 'end'. Your task is to calculate the average duration of user sessions.

data = [ (1, "2022-01-01 10:00", "start"), (1, "2022-01-01 10:15", "end"), (2, "2022-01-01 11:00", "start"), (1, "2022-01-01 11:30", "start"), (2, "2022-01-01 11:45", "end"), (1, "2022-01-01 12:00", "end") ] schema = ["user_id", "timestamp", "action"]



NTILE() window function returns the relative rank of result rows within a window partition. In below example we have used 2 as an argument to ntile hence it returns ranking between 2 values (1 and 2)

RANK() window function is used to provide a rank to the result within a window partition. This function leaves gaps in rank when there are ties.

DENSE_RANK() window function is used to get the result with rank of rows within a window partition without any gaps. This is similar to RANK() function difference being rank function leaves gaps in rank when there are ties.

ROW_NUMBER() window function is used to give the sequential row number starting from 1 to the result of each window partition.

LAG() is a function that works as the offset row returning the value of the before row of a column with respect to the current row.

LEAD() is a function that works as the offset row returning the value of the after row of a column with respect to the current row.

PERCENTILE_RANK() Returns the percentile rank of rows within a window partition.

Here is the Code 👇 :
--------------------


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, row_number, dense_rank, lag, lead, percent_rank, ntile

spark = SparkSession.builder.appName("Window_functions").getOrCreate()

simpleData = (("James", "Sales", 3000),
 ("Michael", "Sales", 4600),
 ("Robert", "Sales", 4100),
 ("Maria", "Finance", 3000),
 ("James", "Sales", 3000),
 ("Scott", "Finance", 3300),
 ("Jen", "Finance", 3900),
 ("Jeff", "Marketing", 3000),
 ("Kumar", "Marketing", 2000),
 ("Saif", "Sales", 4100)
 )
columns = ("name","dept","salary")

#creating a dataframe
df = spark.createDataFrame(simpleData,columns)

#creating a Window specification
window_spec = Window.partitionBy("dept").orderBy("salary")

#Applying window functions
df.withColumn("row_number",row_number().over(window_spec))\
 .withColumn("rank",rank().over(window_spec))\
 .withColumn("dense_rank",dense_rank().over(window_spec))\
 .withColumn("percent_rank",percent_rank().over(window_spec))\
 .withColumn("lag",lag("salary",2).over(window_spec))\
 .withColumn("lead",lead("salary",2).over(window_spec))\
 .withColumn("ntile",ntile(2).over(window_spec))\
 .show()

+-------+---------+------+----------+----+----------+------------+----+----+-----+
|   name|     dept|salary|row_number|rank|dense_rank|percent_rank| lag|lead|ntile|
+-------+---------+------+----------+----+----------+------------+----+----+-----+
|  Maria|  Finance|  3000|         1|   1|         1|         0.0|null|3900|    1|
|  Scott|  Finance|  3300|         2|   2|         2|         0.5|null|null|    1|
|    Jen|  Finance|  3900|         3|   3|         3|         1.0|3000|null|    2|
|  Kumar|Marketing|  2000|         1|   1|         1|         0.0|null|null|    1|
|   Jeff|Marketing|  3000|         2|   2|         2|         1.0|null|null|    2|
|  James|    Sales|  3000|         1|   1|         1|         0.0|null|4100|    1|
|  James|    Sales|  3000|         2|   1|         1|         0.0|null|4100|    1|
| Robert|    Sales|  4100|         3|   3|         2|         0.5|3000|4600|    1|
|   Saif|    Sales|  4100|         4|   3|         2|         0.5|3000|null|    2|
|Mic